In [ ]:
import pandas as pd

In [3]:
df_fichas2022 = pd.read_parquet('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/2022/fichas2022.parquet')
df_voto2022 = pd.read_parquet('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/2022/votomunicipio2022.parquet')
df_municipio = pd.read_parquet('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/diversos/municipio.parquet')

In [4]:
# Ajustes nos dados de 2022
df_fichas2022 = df_fichas2022[df_fichas2022['ds_sit_urna'] == 'Consta da urna']
df_fichas2022 = df_fichas2022[df_fichas2022['sit_partido'].isin (['Deferido', 'Deferido com recurso'])]
df_fichas2022 = df_fichas2022[df_fichas2022['sit_cand'].isin (['Deferido', 'Deferido com recurso'])]
df_fichas2022['ds_eleicao'] = df_fichas2022['ds_eleicao'].replace({
    'Eleito por média': 'Eleito',
    'Eleito por QP': 'Eleito',
    'Suplente' : 'Não eleito',
    'Concorrendo' : 'Não eleito',
})
df_fichas2022['nm_cargo'] = df_fichas2022['nm_cargo'].replace({'Deputado Distrital': 'Deputado Estadual'})
#Atualização manual de dados
#df_fichas2022.loc[df_fichas2022['nu_titulo_eleitor'] == '026474760906', 'ds_eleicao'] = 'Eleito' #Incluir GEOVANIA DE SA na lista de eleitos
#df_fichas2022.loc[df_fichas2022['nu_titulo_eleitor'] == '235392470124', 'ds_eleicao'] = 'Não Eleito' # Retirada do dep. Estadual VINICIUS CAMARINHA agora e prefeito
#df_fichas2022.loc[df_fichas2022['nu_titulo_eleitor'] == '090298850531', 'ds_eleicao'] = 'Não Eleito' # Retirada do dep. Estadual PABLO ROBERTO GONÇALVES DA SILVA agora e vice-prefeito



In [5]:
df_fichas2022 = df_fichas2022[['id_cand','uf_cand','nm_municipio','nm_cand','sg_partido','nm_cargo','ds_eleicao','ds_raca','ds_genero','nu_titulo_eleitor','li_foto']]

df_fichas2022 = df_fichas2022.rename(columns={
    'uf_cand' : 'uf_candidatura',
    'nm_municipio' : 'estado_candidatura'
})

In [6]:
colunas = df_voto2022['eleicao'].unique().tolist()


In [ ]:
df_voto2022 = df_voto2022[df_voto2022['turno'] == 1]
df_voto2022 = df_voto2022[['nu_titulo_eleitor', 'municipio', 'votos']]
df_municipio = df_municipio[['id','uf' ,'nome','capital','no_regiao_brasil']]

In [8]:
df_eleicao2022 = pd.merge(
    df_voto2022, 
    df_municipio,
    left_on='municipio',
    right_on= 'id',
    how='inner')

In [9]:
df_eleicao2022_completo = pd.merge(
    df_fichas2022,
    df_eleicao2022,    
    left_on='nu_titulo_eleitor',
    right_on='nu_titulo_eleitor',
    how='right')

In [10]:
df_eleicao2022_completo.head(5) 

,id_cand,uf_candidatura,estado_candidatura,nm_cand,sg_partido,nm_cargo,ds_eleicao,ds_raca,ds_genero,nu_titulo_eleitor,li_foto,municipio,votos,id,uf,nome,capital,no_regiao_brasil
0,1.000162e+10,AC,ACRE,JORGE NEY VIANA MACEDO NEVES,PT,Governador,Não eleito,PARDA,MASC.,000780342402,https://datarepo.psdb.org.br/fotos/10001616169...,1007,1670,1007,AC,BUJARI,False,Norte
1,1.000164e+10,AC,ACRE,CYLMARA FERNANDES DA ROCHA GRIPP,MDB,Governador,Não eleito,PARDA,FEM.,002324422437,https://datarepo.psdb.org.br/fotos/10001639519...,1007,946,1007,AC,BUJARI,False,Norte
2,1.000160e+10,AC,ACRE,SÉRGIO DE OLIVEIRA CUNHA,PSD,Governador,Não eleito,PARDA,MASC.,000825962445,https://datarepo.psdb.org.br/fotos/10001601556...,1007,264,1007,AC,BUJARI,False,Norte
3,1.000161e+10,AC,ACRE,DAVID SOARES HALL,AGIR,Governador,Não eleito,PARDA,MASC.,004409242488,https://datarepo.psdb.org.br/fotos/10001606307...,1007,8,1007,AC,BUJARI,False,Norte
4,1.000162e+10,AC,ACRE,MARCIO MIGUEL BITTAR,UNIÃO,Governador,Não eleito,PARDA,MASC.,000259632429,https://datarepo.psdb.org.br/fotos/10001623440...,1007,80,1007,AC,BUJARI,False,Norte


In [11]:
df_candidatos_votos = (
    df_eleicao2022_completo.groupby(['uf_candidatura', 'estado_candidatura','nu_titulo_eleitor', 'nm_cand', 'sg_partido', 'nm_cargo', 'ds_eleicao'])['votos']
    .sum()
    .reset_index()
)

In [12]:
df_candidatos_votos.head(5)

,uf_candidatura,estado_candidatura,nu_titulo_eleitor,nm_cand,sg_partido,nm_cargo,ds_eleicao,votos
0,AC,ACRE,000003992496,FRANCISCO DAS CHAGAS BATISTA LOPES,PSB,Deputado Estadual,Não eleito,1438
1,AC,ACRE,000008632402,FRANCIMAR FERNANDES DE ALBUQUERQUE,PT,Deputado Federal,Não eleito,3893
2,AC,ACRE,000023342453,FRANCISCO DA SILVA PIYÃKO,PSD,Deputado Federal,Não eleito,2206
3,AC,ACRE,000041802879,MARIO MARCELO BARROS DOS SANTOS,PSD,Deputado Estadual,Não eleito,1384
4,AC,ACRE,000056352496,OTON RODRIGUES SALES,UNIÃO,Deputado Estadual,Não eleito,822


In [13]:
df_eleicao2022_completo.head(2)

,id_cand,uf_candidatura,estado_candidatura,nm_cand,sg_partido,nm_cargo,ds_eleicao,ds_raca,ds_genero,nu_titulo_eleitor,li_foto,municipio,votos,id,uf,nome,capital,no_regiao_brasil
0,1.000162e+10,AC,ACRE,JORGE NEY VIANA MACEDO NEVES,PT,Governador,Não eleito,PARDA,MASC.,000780342402,https://datarepo.psdb.org.br/fotos/10001616169...,1007,1670,1007,AC,BUJARI,False,Norte
1,1.000164e+10,AC,ACRE,CYLMARA FERNANDES DA ROCHA GRIPP,MDB,Governador,Não eleito,PARDA,FEM.,002324422437,https://datarepo.psdb.org.br/fotos/10001639519...,1007,946,1007,AC,BUJARI,False,Norte


In [14]:
df_estado_votos = (
    df_eleicao2022_completo.groupby(['uf_candidatura', 'estado_candidatura','nu_titulo_eleitor', 'nm_cand', 'sg_partido', 'nm_cargo', 'uf','ds_eleicao'])['votos']
    .sum()
    .reset_index()
)

In [15]:
df_estado_votos

,uf_candidatura,estado_candidatura,nu_titulo_eleitor,nm_cand,sg_partido,nm_cargo,uf,ds_eleicao,votos
0,AC,ACRE,000003992496,FRANCISCO DAS CHAGAS BATISTA LOPES,PSB,Deputado Estadual,AC,Não eleito,1438
1,AC,ACRE,000008632402,FRANCIMAR FERNANDES DE ALBUQUERQUE,PT,Deputado Federal,AC,Não eleito,3893
2,AC,ACRE,000023342453,FRANCISCO DA SILVA PIYÃKO,PSD,Deputado Federal,AC,Não eleito,2206
3,AC,ACRE,000041802879,MARIO MARCELO BARROS DOS SANTOS,PSD,Deputado Estadual,AC,Não eleito,1384
4,AC,ACRE,000056352496,OTON RODRIGUES SALES,UNIÃO,Deputado Estadual,AC,Não eleito,822
...,...,...,...,...,...,...,...,...,...
25752,TO,TOCANTINS,180255490256,TIAGO SOARES SILVA,PRTB,Deputado Estadual,TO,Não eleito,41
25753,TO,TOCANTINS,195763980108,GERSON ALVES DE OLIVEIRA,PSB,Deputado Estadual,TO,Não eleito,410
25754,TO,TOCANTINS,223164560191,EDUARDO MALHEIRO RIBEIRO FORTES,PSD,Deputado Estadual,TO,Eleito,15466
25755,TO,TOCANTINS,272998690191,RICARDO IGNACIO DE MACEDO,PMB,Governador,TO,Não eleito,5043


In [16]:
df_candidatos_votos.loc[df_candidatos_votos['nm_cand'] == 'LUIZ INÁCIO LULA DA SILVA']

,uf_candidatura,estado_candidatura,nu_titulo_eleitor,nm_cand,sg_partido,nm_cargo,ds_eleicao,votos
3399,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,Eleito,57259504


In [17]:
df_estado_votos.loc[df_estado_votos['nm_cand'] == 'LUIZ INÁCIO LULA DA SILVA']

,uf_candidatura,estado_candidatura,nu_titulo_eleitor,nm_cand,sg_partido,nm_cargo,uf,ds_eleicao,votos
3588,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,AC,Eleito,129022
3589,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,AL,Eleito,974156
3590,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,AM,Eleito,1019684
3591,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,AP,Eleito,197382
3592,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,BA,Eleito,5873081
3593,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,CE,Eleito,3578355
3594,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,DF,Eleito,649534
3595,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,ES,Eleito,897348
3596,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,GO,Eleito,1454723
3597,BR,BRASIL,122418060191,LUIZ INÁCIO LULA DA SILVA,PT,Presidente,MA,Eleito,2603454


In [ ]:
#df_votos_candidatos.to_excel('/home/filipe/scripts/psdb-projetos/dashStreamlit/datasets/2022/teste_votos 2.xlsx', index=False)

: 